In [ ]:

!pip install -q transformers accelerate datasets peft bitsandbytes


In [ ]:

import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model

# -----------------------------
# 1) تنظیمات کلی
# -----------------------------
BASE_MODEL = "Qwen/Qwen2-0.5B"   # می‌توانی اینجا مدل را عوض کنی
OUTPUT_DIR = "./qwen-lora-finetuned"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

# -----------------------------
# 2) لود توکنایزر و مدل پایه
# -----------------------------
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True,
    load_in_8bit=True,          # اگر GPU ضعیف است کمک می‌کند
    device_map="auto"
)

# -----------------------------
# 3) تعریف LoRA روی مدل
# -----------------------------
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# -----------------------------
# 4) آماده کردن دیتاست نمونه
#    اینجا فقط چند جمله‌ی تستی گذاشته شده.
#    در عمل باید دیتاست خودت را جایگزین کنی.
# -----------------------------

texts = [
    "سوال: جمع ۲ و ۲ چند می‌شود؟ جواب: ۴ است.",
    "سوال: پایتورچ برای چه استفاده می‌شود؟ جواب: برای ساخت و آموزش شبکه‌های عصبی.",
    "سوال: RL چیست؟ جواب: یک چارچوب یادگیری بر اساس پاداش و تنبیه.",
]

raw_dataset = Dataset.from_dict({"text": texts})

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,
        padding="max_length",
    )

tokenized_dataset = raw_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

# -----------------------------
# 5) دیتاکولاتور برای زبان‌مدل‌سازی
# -----------------------------
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# -----------------------------
# 6) تنظیمات آموزش
# -----------------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=50,
    save_total_limit=2,
    warmup_ratio=0.03,
    weight_decay=0.01,
    report_to="none",
)

# -----------------------------
# 7) ساخت Trainer و شروع آموزش
# -----------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

trainer.train()

# -----------------------------
# 8) ذخیره LoRA adapter
# -----------------------------
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("✅ فاین‌تیون تمام شد و LoRA adapter در", OUTPUT_DIR, "ذخیره شد.")


In [ ]:

# مثال: استفاده از مدل فاین‌تیون‌شده برای اینفرنس

from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

BASE_MODEL = "Qwen/Qwen2-0.5B"
ADAPTER_DIR = "./qwen-lora-finetuned"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True,
    device_map="auto",
    load_in_8bit=True,
)

model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
model.eval()

prompt = "سوال: RL چیست؟ جواب:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        top_p=0.9,
        temperature=0.7,
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
